In [122]:
import pandas as pd
import numpy as np
import warnings
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [123]:
df = pd.read_csv('games_may2024_cleaned.csv')
df.head(3)

,AppID,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,...,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags,pct_pos_total,num_reviews_total,pct_pos_recent,num_reviews_recent
0,730,Counter-Strike 2,2012-08-21,0,0.0,1,"For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...","For over two decades, Counter-Strike has offer...",NaN,...,29018,753,6078,352,1362469,"{'FPS': 90076, 'Shooter': 64786, 'Multiplayer'...",87,8071426,80,56072
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.0,0,"LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...","LAND, LOOT, SURVIVE! Play PUBG: BATTLEGROUNDS ...",Play PUBG: BATTLEGROUNDS for free. Land on str...,NaN,...,0,0,0,0,590582,"{'Survival': 14611, 'Shooter': 12477, 'Battle ...",58,2365012,68,19517
2,570,Dota 2,2013-07-09,0,0.0,2,"The most-played game on Steam. Every day, mill...","The most-played game on Steam. Every day, mill...","Every day, millions of players worldwide enter...",“A modern multiplayer masterpiece.” 9.5/10 – D...,...,35557,1327,1163,977,668192,"{'Free to Play': 59559, 'MOBA': 19966, 'Multip...",81,2247365,71,23832


In [124]:
df.drop(['score_rank','metacritic_url','reviews','detailed_description','about_the_game','short_description','header_image','website','support_url','support_email','tags'],axis=1,inplace=True)

In [125]:
df['release_date'] = pd.to_datetime(df['release_date'], format='mixed')
df['year'] = df['release_date'].dt.year
df['month'] = df['release_date'].dt.month
df['day'] = df['release_date'].dt.day

In [126]:
df.columns

Index(['AppID', 'name', 'release_date', 'required_age', 'price', 'dlc_count',
       'windows', 'mac', 'linux', 'metacritic_score', 'achievements',
       'recommendations', 'notes', 'supported_languages',
       'full_audio_languages', 'packages', 'developers', 'publishers',
       'categories', 'genres', 'screenshots', 'movies', 'user_score',
       'positive', 'negative', 'estimated_owners', 'average_playtime_forever',
       'average_playtime_2weeks', 'median_playtime_forever',
       'median_playtime_2weeks', 'peak_ccu', 'pct_pos_total',
       'num_reviews_total', 'pct_pos_recent', 'num_reviews_recent', 'year',
       'month', 'day'],
      dtype='object')

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83646 entries, 0 to 83645
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   AppID                     83646 non-null  int64         
 1   name                      83643 non-null  object        
 2   release_date              83646 non-null  datetime64[ns]
 3   required_age              83646 non-null  int64         
 4   price                     83646 non-null  float64       
 5   dlc_count                 83646 non-null  int64         
 6   windows                   83646 non-null  bool          
 7   mac                       83646 non-null  bool          
 8   linux                     83646 non-null  bool          
 9   metacritic_score          83646 non-null  int64         
 10  achievements              83646 non-null  int64         
 11  recommendations           83646 non-null  int64         
 12  notes             

In [128]:
df['genres'] = df['genres'].apply(eval)

In [129]:
genres_expanded = pd.DataFrame(df['genres'].to_list(), columns=[f'Genre_{i+1}' for i in range(df['genres'].str.len().max())])


data = pd.concat([df, genres_expanded], axis=1)

In [134]:
data[['lower_bound', 'upper_bound']] = data['estimated_owners'].str.split(' - ', expand=True)

data['lower_bound'] = data['lower_bound'].astype(int)
data['upper_bound'] = data['upper_bound'].astype(int)

data['average_owners'] = (data['lower_bound'] + data['upper_bound']) / 2
data = data.drop(columns=['lower_bound', 'upper_bound'])

In [135]:
data['estimated_revenue'] = data['price'] * data['average_owners']
data['median_playtime_forever_in_hours'] = data['median_playtime_forever'] / 60
data['average_playtime_forever_in_hours'] = data['average_playtime_forever'] / 60

In [136]:
data['month_year'] = data['release_date'].apply(lambda x: x.strftime('%B-%Y'))

In [137]:
data['name'].isnull().sum()

3

In [138]:
data.isnull().sum()

AppID                                0
name                                 3
release_date                         0
required_age                         0
price                                0
                                    ..
average_owners                       0
estimated_revenue                    0
median_playtime_forever_in_hours     0
average_playtime_forever_in_hours    0
month_year                           0
Length: 62, dtype: int64

In [139]:
data

,AppID,name,release_date,required_age,price,dlc_count,windows,mac,linux,metacritic_score,...,Genre_15,Genre_16,Genre_17,Genre_18,Genre_19,average_owners,estimated_revenue,median_playtime_forever_in_hours,average_playtime_forever_in_hours,month_year
0,730,Counter-Strike 2,2012-08-21,0,0.00,1,True,False,True,0,...,None,None,None,None,None,150000000.0,0.0,101.300000,483.633333,August-2012
1,578080,PUBG: BATTLEGROUNDS,2017-12-21,0,0.00,0,True,False,False,0,...,None,None,None,None,None,75000000.0,0.0,0.000000,0.000000,December-2017
2,570,Dota 2,2013-07-09,0,0.00,2,True,True,True,90,...,None,None,None,None,None,350000000.0,0.0,19.383333,592.616667,July-2013
3,271590,Grand Theft Auto V,2015-04-13,17,0.00,1,True,False,False,96,...,None,None,None,None,None,75000000.0,0.0,115.166667,214.966667,April-2015
4,359550,Tom Clancy's Rainbow Six® Siege,2015-12-01,17,19.99,9,True,False,False,0,...,None,None,None,None,None,35000000.0,699650000.0,50.266667,270.216667,December-2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83641,1745960,Circuit Breaker,2021-10-21,0,4.99,1,True,False,False,0,...,None,None,None,None,None,10000.0,49900.0,0.000000,0.000000,October-2021
83642,1888920,Chess vs Chat,2022-08-09,0,3.99,0,True,False,False,0,...,None,None,None,None,None,10000.0,39900.0,0.000000,0.000000,August-2022
83643,2166420,Orbtangle,2022-10-28,0,1.99,0,True,False,False,0,...,None,None,None,None,None,10000.0,19900.0,0.000000,0.000000,October-2022
83644,1456260,Flashlight,2021-10-25,0,2.99,0,True,True,False,0,...,None,None,None,None,None,10000.0,29900.0,0.000000,0.000000,October-2021


In [140]:
data.to_csv("games_prepped.csv")

In [141]:
data = pd.read_csv('games_prepped.csv')

In [144]:
data.dtypes

Unnamed: 0                             int64
AppID                                  int64
name                                  object
release_date                          object
required_age                           int64
                                      ...   
average_owners                       float64
estimated_revenue                    float64
median_playtime_forever_in_hours     float64
average_playtime_forever_in_hours    float64
month_year                            object
Length: 63, dtype: object